   # MISE A JOUR BASE DE DONNEES SPECTRALES UVEX

## Veuillez choisir un fichier à uploader dans la base 

In [2]:
import mysql.connector
import datetime 
import os
import numpy as np
from astropy.wcs import WCS
from astropy.io import fits
from mysql.connector import errorcode
from ftplib import FTP
from tkinter import filedialog 
from tkinter import *
from io import BytesIO 
    
root = Tk() 
root.filename= filedialog.askopenfilename (initialdir = "X:/spectres/" , title = "Select file" , filetypes = [( "fit files" , "*.fit" ), ( "fits files" , " *. fits" )]) 
print (" Fichier sélectionné : " + root.filename)
file=root.filename
root.destroy()

 Fichier sélectionné : X:/ETOILES BE/phi Per/phi Per_20200214_IR.fit


## Entête fits du fichier selectionné

In [3]:
sp = fits.open(file)
print(sp.info())
header = sp[0].header

for cle,valeur in header.items():
    print(str(cle)+ " : " + str(valeur))

if not('BSS_ESRP' in header) :
    header['BSS_ESRP']=0
if not('BSS_SRPW' in header) :
    header['BSS_SRPW']=0
if not('BSS_BINN' in header) :
    header['BSS_BINN']=1
if not('BSS_VMAG' in header) :
    header['BSS_VMAG']=0
if not('BSS_STYP' in header) :
    header['BSS_STYP']=''
if not('BSS_LAT' in header) :
    header['BSS_LAT']=0
if not('BSS_LONG' in header) :
    header['BSS_LONG']=0
if not('BSS_ELEV' in header) :
    header['BSS_ELEV']=0
if not('TELESCOP' in header) :
    header['TELESCOP']=''
if not('DETNAM' in header) :
    header['DETNAM']=''
if not('INSTRUMEN' in header) :
    header['INSTRUMEN']=''
if not('RA' in header) :
    header['RA']=''
if not('DEC' in header) :
    header['DEC']=''
    
wcs = WCS(header)


Filename: X:/ETOILES BE/phi Per/phi Per_20200214_IR.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      30   (20613,)   float32   
None
SIMPLE : True
BITPIX : -32
NAXIS : 1
NAXIS1 : 20613
CRVAL1 : 7565.629393
CDELT1 : 0.09085995
VERSION : ISIS V5.9.7
OBJNAME : Phi Per
DATE-OBS : 2020-02-14T22:56:36
EXPTIME : -21
EXPTIME2 : 10 x 300 s
BSS_INST : LX200GPS 12" + UVEX 600 tr/mm blaze 750 nm + ASI 183
BSS_SITE : Observatoire du Thiey
BSS_ITRP : 3106
OBSERVER : JL MARTIN
CUNIT1 : Angstrom
CTYPE1 : Wavelength
Q_CAL : 0
CRPIX1 : 1
BSS_VHEL : 0
BSS_COSM : Removed
BSS_TELL : None
BSS_NORM : None
SPE_RPOW : 3106
JD-OBS : 2458894.456
JD-MID : 2458894.4559
JD-HEL : 0
GEO_LONG : 1.50858
GEO_LAT : 43.51728
GEO_ELEV : 150


## Insertion dans la base de données UVEX 

In [5]:
filename= datetime.datetime.today().strftime('%Y-%m-%d')+'_'+header['OBJNAME']+'_'+header['DATE-OBS']+".fit"
print("Nom de fichier sauvegardé dans BD : " + filename)

try:
    #cnx=mysql.connector.connect(user='root',password='',host='127.0.0.1',database='bd_uvex')
    #cnx=mysql.connector.connect(user='root@localhost',password='Malibert006@',host='192.168.1.7',database='BD_UVEX',port='3307')
    cnx=mysql.connector.connect(user='u429156696_uvex30',password='Malibert006@',host='sql289.main-hosting.eu',database='u429156696_bd_uvex')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:  
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
H=header
mycursor=cnx.cursor()
sql="insert into SPECTRE (DATE_OBS,EXPTIME,EXPTIME2,CRVAL1,CDELT1,CRPIX1,CUNIT1,CTYPE1,BSS_ESRP,BSS_ITRP,BSS_SRPW,BSS_BINN,BSS_VMAG,BSS_STYP,BSS_SITE,BSS_LAT,BSS_LONG,BSS_ELEV,TELESCOP,DETNAM,INSTRUMEN,OBSERVER,RA,DECLI,OBJNAME,JD_MID,JD_HEL,JD_OBS,FILENAME) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
val=(H['DATE-OBS'],H['EXPTIME'],H['EXPTIME2'],H['CRVAL1'],H['CDELT1'],H['CRPIX1'],H['CUNIT1'],H['CTYPE1'],H['BSS_ESRP'],H['BSS_ITRP'],H['BSS_SRPW'],H['BSS_BINN'],H['BSS_VMAG'],H['BSS_STYP'],H['BSS_SITE'],H['BSS_LAT'],H['BSS_LONG'],H['BSS_ELEV'],H['TELESCOP'],H['DETNAM'],H['INSTRUMEN'],H['OBSERVER'],H['RA'],H['DEC'],H['OBJNAME'],H['JD-MID'],H['JD-HEL'],H['JD-OBS'],filename)
mycursor.execute(sql,val)
cnx.commit()
print("Insertion dans la BD effectuée")

host = "81.16.28.179" # adresse du serveur FTP
user = "u429156696" # votre identifiant
password = "Malibert006@" # votre mot de passe
ftp=FTP(host=host)
ftp.login(user=user,passwd=password)
fichier = open(file, 'rb') 
ftp.storbinary('STOR '+ '/BD_SPECTRES/'+filename,fichier)   
fichier.close()
print('Fichier uploadé ')


Nom de fichier sauvegardé dans BD : 2020-04-01_Phi Per_2020-02-14T22:56:36.fit
Insertion dans la BD effectuée
Fichier uploadé 
